# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dudinka,69.4058,86.1778,-2.38,98,82,10.31,RU,1699315511
1,1,mchinji,-13.7984,32.8802,69.49,70,40,3.71,MW,1699315748
2,2,namibe,-15.1961,12.1522,66.79,69,39,3.44,AO,1699315749
3,3,the pas,53.8251,-101.2541,27.05,79,99,7.07,CA,1699315750
4,4,east london,-33.0153,27.9116,66.56,94,100,17.09,ZA,1699315752


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
import folium
import random

# Create a base map
m = folium.Map(location=[city_data_df['Lat'].mean(), city_data_df['Lng'].mean()], zoom_start=2)

# Create a custom legend
legend_html = """
<div style="position: fixed; 
     bottom: 50px; right: 50px; width: 250px; 
     border: 2px solid grey; z-index: 9999; font-size: 14px;
     background-color: white;
     opacity: 0.8;
     max-height: 250px;  /* Set maximum height */
     overflow-y: auto;  /* Enable vertical scrolling to account for all the cities */
     ">
     <b>City Legend</b><br>
"""

# Define a function to generate a random color
def get_random_color():
    return "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# Add markers to the map for each city, using different colors for visiblity 
for i, row in city_data_df.iterrows():
    color = get_random_color()
    
    # Add the circle marker to the map
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 15, # so the marker doesn't completely block out the city
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.4,
        popup=f"{row['City']}, Humidity: {row['Humidity']}"
    ).add_to(m)

    # Add the legend entry for each city with circular marker
    legend_html += f'<span style="display: inline-block; width: 20px; height: 20px; background-color: {color}; border: 1px solid grey;"></span>{row["City"]}<br>'

# Complete the legend HTML
legend_html += "</div>"

# Add the legend HTML to the map
m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[(city_data_df['Max Temp'] > 24) & 
                               (city_data_df['Max Temp'] < 36) & 
                               (city_data_df['Humidity'] < 80)& 
                               (city_data_df['Cloudiness'] == 0)].copy()

# Drop any rows with null values (if any)
ideal_cities_df.dropna(inplace=True)

# Display the sample data of ideal cities
ideal_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
387,387,gaspe,48.8334,-64.4819,30.11,68,0,3.44,CA,1699316222
546,546,yelizovo,53.1873,158.3805,24.94,68,0,18.01,RU,1699316460


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
387,gaspe,CA,48.8334,-64.4819,68,
546,yelizovo,RU,53.1873,158.3805,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
import json

# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
params = {
    "filter": "location",
    "bias": "proximity"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params, headers={"x-geapify-api-key": geoapify_key}).json()
    
    # Convert the API response to JSON format
    name_address = json.loads(json.dumps(name_address))
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
gaspe - nearest hotel: No hotel found
yelizovo - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
387,gaspe,CA,48.8334,-64.4819,68,No hotel found
546,yelizovo,RU,53.1873,158.3805,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
m = folium.Map(location=[hotel_df['Lat'].mean(), hotel_df['Lng'].mean()], zoom_start=2)

# Add markers to the map for each city's hotel
for index, row in hotel_df.iterrows():
    # Create a popup content with the desired information
    popup_content = f"City: {row['City']}<br>Country: {row['Country']}<br>Hotel: {row['Hotel Name']}<br>"
    popup_content += f"Latitude: {row['Lat']}<br>Longitude: {row['Lng']}<br>Humidity: {row['Humidity']}"

    # Check if a hotel was found
    if row["Hotel Name"] != "No hotel found":
        folium.Marker(
            location=[row['Lat'], row['Lng']],
            popup=folium.Popup(popup_content),
            icon=folium.Icon(color='green')
        ).add_to(m)
    else:
        # If no hotel was found, create a red marker
        folium.Marker(
            location=[row['Lat'], row['Lng']],
            popup=folium.Popup(popup_content),
            icon=folium.Icon(color='red')
        ).add_to(m)

# Display the map
m
